In [59]:
import pandas as pd
def calculate_distance_matrix(df)->pd.DataFrame():
    """
    Calculate a distance matrix based on the dataframe, df.

    Args:
        df (pandas.DataFrame)

    Returns:
        pandas.DataFrame: Distance matrix
    """
    # Write your logic here
     # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file_path)

    # Create a pivot table to get distances between toll locations
    distance_matrix = df.pivot(index='id_start', columns='id_end', values='distance').fillna(0)

    # Convert the pivot table to a DataFrame
    distance_matrix = pd.DataFrame(distance_matrix)

    # Iterate through the DataFrame and calculate cumulative distances
    for column in distance_matrix.columns:
        for row in distance_matrix.index:
            if row != column and distance_matrix.at[row, column] == 0:
                # If there is no direct route, calculate cumulative distance
                row_distances = distance_matrix.loc[row]
                cumulative_distance = row_distances[row_distances != 0].sum()
                distance_matrix.at[row, column] = cumulative_distance
                distance_matrix.at[column, row] = cumulative_distance

    # Set diagonal values to 0
    for idx in distance_matrix.index:
        distance_matrix.at[idx, idx] = 0

    return distance_matrix
csv_file_path = 'dataset-3.csv'
result_matrix = calculate_distance_matrix(csv_file_path)
print(result_matrix)



id_end    1001402  1001404  1001406  1001408  1001410  1001412  1001414  \
id_start                                                                  
1001400       9.7      9.7     19.4     38.8     77.6    155.2    310.4   
1001402       0.0     16.0     21.7     11.1     15.6     18.2     13.2   
1001404      16.0      0.0     43.4     22.2     31.2     36.4     26.4   
1001406      21.7     43.4      0.0     44.4     62.4     72.8     52.8   
1001408      11.1     22.2     44.4      0.0    124.8    145.6    105.6   
1001410      15.6     31.2     62.4    124.8      0.0    291.2    211.2   
1001412      18.2     36.4     72.8    145.6    291.2      0.0    422.4   
1001414      13.2     26.4     52.8    105.6    211.2    422.4      0.0   
1001416      13.6     27.2     54.4    108.8    217.6    435.2    870.4   
1001418      12.9     25.8     51.6    103.2    206.4    412.8    825.6   
1001420       9.6     19.2     38.4     76.8    153.6    307.2    614.4   
1001422      11.4     22.

In [63]:
import pandas as pd

def unroll_distance_matrix(input_matrix):
    # Ensure the input matrix is symmetric
    input_matrix = (input_matrix + input_matrix.T) / 2

    # Extract unique toll locations from the matrix
    toll_locations = input_matrix.index

    # Initialize a list to store unrolled data
    unrolled_data = []

    # Iterate through toll locations and create combinations
    for start_loc in toll_locations:
        for end_loc in toll_locations:
            if start_loc != end_loc:
                distance = round(input_matrix.at[start_loc, end_loc],1)
                unrolled_data.append({'id_start': start_loc, 'id_end': end_loc, 'distance': distance})

    # Create a DataFrame from the unrolled data
    unrolled_df = pd.DataFrame(unrolled_data)
    
    return unrolled_df


# Assuming result_matrix is the DataFrame from Question 1
result_matrix = calculate_distance_matrix('dataset-3.csv')  # You can replace this with the actual path
unrolled_result = unroll_distance_matrix(result_matrix)
print(unrolled_result)


      id_start   id_end      distance
0      1001400  1001402           NaN
1      1001400  1001404  9.700000e+00
2      1001400  1001406  1.940000e+01
3      1001400  1001408  3.880000e+01
4      1001400  1001410  7.760000e+01
...        ...      ...           ...
1801   1004356  1001470  1.374390e+11
1802   1004356  1001472  2.748779e+11
1803   1004356  1001488  5.497558e+11
1804   1004356  1004354  3.958242e+12
1805   1004356  1004355  1.099512e+12

[1806 rows x 3 columns]


In [64]:
import pandas as pd

def find_ids_within_ten_percentage_threshold(input_df, reference_value):
    
    # Filter DataFrame for the given reference_value in id_start
    reference_data = input_df[input_df['id_start'] == reference_value]
    
    # Calculate the average distance for the reference_value
    avg_distance = reference_data['distance'].mean()
   

    # Calculate the lower and upper thresholds within 10%
    lower_threshold = avg_distance - (avg_distance * 0.1)
    upper_threshold = avg_distance + (avg_distance * 0.1)

    # Filter DataFrame for values within the 10% threshold
    within_threshold_data = input_df[(input_df['distance'] >= lower_threshold) & (input_df['distance'] <= upper_threshold)]

    # Extract unique values from id_start column and sort the list
    result_ids = sorted(within_threshold_data['id_start'].unique())

    return result_ids


reference_value = 1001400  # Replace with the desired reference value
result_list = find_ids_within_ten_percentage_threshold(unrolled_result, reference_value)
print(result_list)


[1001464, 1001468, 1001470, 1001472, 1001488, 1004354, 1004356]


In [56]:
def calculate_toll_rate(input_df):
    # Define rate coefficients for each vehicle type
    rate_coefficients = {'moto': 0.8, 'car': 1.2, 'rv': 1.5, 'bus': 2.2, 'truck': 3.6}

    # Iterate through vehicle types and calculate toll rates
    for vehicle_type, rate_coefficient in rate_coefficients.items():
        column_name = f"{vehicle_type}_rate"
        input_df[column_name] = input_df['distance'] * rate_coefficient

    return input_df


# Assuming unrolled_result is the DataFrame from Question 2
result_with_rates = calculate_toll_rate(unrolled_result)
print(result_with_rates)


      id_start   id_end      distance     moto_rate      car_rate  \
0      1001400  1001402           NaN           NaN           NaN   
1      1001400  1001404  9.700000e+00  7.760000e+00  1.164000e+01   
2      1001400  1001406  1.940000e+01  1.552000e+01  2.328000e+01   
3      1001400  1001408  3.880000e+01  3.104000e+01  4.656000e+01   
4      1001400  1001410  7.760000e+01  6.208000e+01  9.312000e+01   
...        ...      ...           ...           ...           ...   
1801   1004356  1001470  1.374390e+11  1.099512e+11  1.649267e+11   
1802   1004356  1001472  2.748779e+11  2.199023e+11  3.298535e+11   
1803   1004356  1001488  5.497558e+11  4.398047e+11  6.597070e+11   
1804   1004356  1004354  3.958242e+12  3.166593e+12  4.749890e+12   
1805   1004356  1004355  1.099512e+12  8.796093e+11  1.319414e+12   

           rv_rate      bus_rate    truck_rate  
0              NaN           NaN           NaN  
1     1.455000e+01  2.134000e+01  3.492000e+01  
2     2.910000e+01  4.26